In [1]:
import geopandas as gpd
import pandas as pd

# Select background tiles

In [2]:
# To use satellite imagery, the `xyzserives` must be installed
try:
    import xyzservices

    tiles = xyzservices.providers["Esri"]["WorldImagery"]
except:
    # Use default
    tiles = None

# Loading data

In [3]:
df_selected_bursts = pd.read_csv(
    "selected_bursts_option0.csv.zip", parse_dates=["sensing_date"]
)
df_selected_bursts.sensing_date = df_selected_bursts.sensing_date.dt.date


df_selected_bursts.head(3)

,frame_id,total_num_bursts,burst_id_jpl,sensing_date,num_candidate_bursts
0,831,5022,t004_006641_iw1,2017-02-15,5298
1,831,5022,t004_006641_iw1,2017-02-27,5298
2,831,5022,t004_006641_iw1,2017-03-11,5298


In [4]:
df_selected_bursts_grouped = (
    df_selected_bursts[["burst_id_jpl", "frame_id"]]
    .groupby(("burst_id_jpl"))
    .count()
    .rename({"frame_id": "date_count"}, axis=1)
)

df_selected_bursts_grouped.head()

,date_count
burst_id_jpl,
t004_006641_iw1,186
t004_006641_iw2,186
t004_006641_iw3,186
t004_006642_iw1,186
t004_006642_iw2,186


In [5]:
df_burst_geoms = gpd.read_file("na_bursts_with_geom.geojson.zip")
df_burst_geoms.head(3)

,frame_id,burst_id_jpl,orbit_pass,geometry
0,1653,t007_013218_iw1,ASCENDING,"MULTIPOLYGON (((-157.82827 55.48763, -157.1283..."
1,1653,t007_013222_iw3,ASCENDING,"MULTIPOLYGON (((-155.37657 56.54824, -154.7210..."
2,1653,t007_013225_iw2,ASCENDING,"MULTIPOLYGON (((-156.93933 56.84940, -156.1855..."


In [6]:
df_burst_count = pd.merge(df_selected_bursts_grouped, df_burst_geoms, on="burst_id_jpl")
df_burst_count = gpd.GeoDataFrame(df_burst_count, geometry="geometry")
df_burst_count.head()

,burst_id_jpl,date_count,frame_id,orbit_pass,geometry
0,t004_006641_iw1,186,831,ASCENDING,"MULTIPOLYGON (((-77.59315 33.46137, -77.11728 ..."
1,t004_006641_iw2,186,831,ASCENDING,"MULTIPOLYGON (((-76.71825 33.66354, -76.22176 ..."
2,t004_006641_iw3,186,831,ASCENDING,"MULTIPOLYGON (((-75.79204 33.85517, -75.35592 ..."
3,t004_006642_iw1,186,831,ASCENDING,"MULTIPOLYGON (((-77.63169 33.62745, -77.15485 ..."
4,t004_006642_iw2,186,831,ASCENDING,"MULTIPOLYGON (((-76.75500 33.82960, -76.25752 ..."


In [7]:
# Aggregate burst-wise polygons
frame_based = df_burst_count.dissolve(by='frame_id').reset_index()
frame_based["path_number"] = frame_based.burst_id_jpl.str[1:4].astype(int)
frame_based = frame_based.drop(columns='burst_id_jpl')
frame_based

,frame_id,geometry,date_count,orbit_pass,path_number
0,831,"POLYGON ((-77.63462 33.64007, -77.67350 33.807...",186,ASCENDING,4
1,832,"POLYGON ((-77.94234 34.97052, -77.98167 35.136...",387,ASCENDING,4
2,833,"POLYGON ((-78.25450 36.29876, -78.29421 36.463...",402,ASCENDING,4
3,834,"POLYGON ((-78.57216 37.62689, -78.61245 37.791...",399,ASCENDING,4
4,835,"POLYGON ((-78.89565 38.95445, -78.93684 39.119...",394,ASCENDING,4
...,...,...,...,...,...
1428,46541,"POLYGON ((-134.76723 56.39017, -135.42777 56.4...",284,DESCENDING,174
1429,46542,"POLYGON ((-135.18571 55.07265, -135.82487 55.1...",271,DESCENDING,174
1430,46543,"POLYGON ((-135.58757 53.75420, -136.20699 53.8...",264,DESCENDING,174
1431,46799,"MULTIPOLYGON (((-151.98765 70.03128, -153.1816...",2,DESCENDING,175


# View Ascending Frames

In [8]:
frame_based[frame_based.orbit_pass == "ASCENDING"].explore(tiles=tiles, color='red')

# View Descending Frames

In [9]:
frame_based[frame_based.orbit_pass == "DESCENDING"].explore(tiles=tiles)